<a href="https://colab.research.google.com/github/Athulkrishna-S/CerebraLink/blob/main/Model_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os

In [7]:
folder_path = '/content/drive/MyDrive/Final_Project'
file_name='eye_up_down_features.pkl'

In [5]:
from sklearn.model_selection import StratifiedKFold , cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [8]:
data = pd.read_pickle(os.path.join(folder_path,file_name))

In [11]:
data.head()

,mean,std_dev,rms,peak_to_peak,skewness,kurtosis,max_amp,min_amp,dominant_freq,signal_power,hurst_exponent,higuchi_fd,Label
0,187.8132,1869.595227,1879.005086,22230,1.660935,15.285868,11820,-10410,39.0625,4779.170431,1.562065,1.189970,rest
1,367.2804,3564.492020,3583.364097,25800,0.240154,3.465215,13410,-12390,39.0625,14780.217093,1.764543,1.113945,up
2,-14.9826,1903.652736,1903.711695,23490,-2.327222,20.187052,11220,-12270,39.0625,4654.511766,1.566912,1.195824,rest
3,-103.0128,4141.508170,4142.789104,25260,0.891851,2.723926,15210,-10050,39.0625,6620.899303,1.604872,1.192660,down
4,354.7260,1717.732992,1753.977528,18840,2.143470,15.337650,12630,-6210,39.0625,9814.275230,1.407774,1.135784,rest


In [10]:
X=data.drop('Label',axis=1)
Y=data['Label']

In [13]:
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(Y)

In [15]:
print(label_encoder.classes_)

['down' 'rest' 'up']


In [39]:
map = {}

In [40]:
print("Class Mapping:")
for class_label, encoded_label in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    map[encoded_label] = class_label
    print(f"{class_label} -> {encoded_label}")

Class Mapping:
down -> 0
rest -> 1
up -> 2


In [41]:
print(map)

{0: 'down', 1: 'rest', 2: 'up'}


In [16]:
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)

# Set up Stratified K-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [18]:
cv_scores = cross_val_score(rf_model, X, Y, cv=kf, scoring='accuracy')

# Output Cross-Validation Results
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

Cross-Validation Accuracy Scores: [0.95454545 0.84090909 0.88372093 0.95348837 0.86046512]
Mean Accuracy: 0.8986 ± 0.0472


In [20]:
rf_model.fit(X, Y)
y_pred = rf_model.predict(X)

# Generate classification report
print("Classification Report:")
print(classification_report(Y, y_pred, target_names=label_encoder.classes_))

Classification Report:
              precision    recall  f1-score   support

        down       1.00      1.00      1.00        54
        rest       1.00      1.00      1.00       109
          up       1.00      1.00      1.00        54

    accuracy                           1.00       217
   macro avg       1.00      1.00      1.00       217
weighted avg       1.00      1.00      1.00       217



In [28]:
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [48]:
test=data.iloc[3]

In [34]:
test.drop(test.index[-1], inplace=True)  # In-place deletion of the last element
print(test)

mean                -103.0128
std_dev            4141.50817
rms               4142.789104
peak_to_peak            25260
skewness             0.891851
kurtosis             2.723926
max_amp                 15210
min_amp                -10050
dominant_freq         39.0625
signal_power      6620.899303
hurst_exponent       1.604872
higuchi_fd            1.19266
Name: 3, dtype: object


<ipython-input-34-74117a74c931>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(test.index[-1], inplace=True)  # In-place deletion of the last element


In [42]:
#convert the test into numpy 2D array
# you can also convert to DataFrame

test = np.array(test).reshape(1, -1) # 1 means 2D array should have 1 row and -1 means automaticcaly infer number of columns from the test array

# Make prediction
prediction = rf_model.predict(test)
print(prediction)
for i in prediction:
  print(map[i])

[0]
down


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


CONVERT THE PANDAS SERIES test TO DATAFRAME TO AVOID THE WARNING

In [51]:
# Convert the Series to DataFrame with the correct feature names
test_df = pd.DataFrame(test).T  # T transposes the Series into a DataFrame, making it a single row

# Ensure the columns are correctly assigned from the original DataFrame's column names
test_df.columns = data.columns

test_df.drop('Label',axis=1,inplace=True)

# Make the prediction
prediction = rf_model.predict(test_df)
print(prediction)

for i in prediction:
    print(map[i])  # Print the mapped label


[0]
down
